In [1]:
from L420A import everything

In [2]:
everything.setup()

Biosphere database already present!!! No setup is needed
Database has already been imported


In [3]:
everything.constructor_main(file = 'Case_study_existing_building_wEOL_4.xlsx', sheet = 'Construction')

In [4]:
everything.renovation_main(file = 'Case_study_existing_building_wEOL_4.xlsx', sheet = 'Renovation')

In [8]:
for p in everything.Product.real_instances:
    if p.name == "glass wool mat production":
        print(p.major_renovation_possible)

True
False


In [ ]:
mfa = everything.Mfa(2020, 2050)

In [ ]:
for product in everything.Product.instances:
    mfa.add_points(product)

In [ ]:
mfa.points

In [ ]:
mfa.get_max_db_year()

In [ ]:
clean_list = mfa.clean_list_for_lca()

In [ ]:
lca2 = everything.ProLCA(activities_and_years=clean_list, yearly_databases=mfa.create_time_line_dict(), methods=[('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)'), ('EF v3.0 EN15804', 'acidification', 'accumulated exceedance (AE)')])

In [ ]:
lca2.give_me_embodied()

In [ ]:
data = {}
for product in everything.Product.instances:
    product.embodied_impacts = everything.GiveMeName.aggregator(_11, product)
    if not len(product.embodied_impacts) == 0:
        data[product.name] = product.embodied_impacts

In [ ]:
from L420A import everything

In [ ]:
everything.setup()

In [ ]:
# data
# pickle this data

# Path: test.ipynb

import pickle
# with open('data.pickle', 'wb') as handle:
#     pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('data.pickle', 'rb') as handle:
    b = pickle.load(handle)
    

In [ ]:
(b)

In [ ]:
everything.GiveMeName.poop_to_excel(b, 'test.xlsx')

In [ ]:
db = everything.bd.Database('ecoinvent-3.9.1-cuttoff')

In [ ]:
for a in db:
    if a['code'] == '9fda2f5ef0c1309141b798483b169b23':
        print(a)

In [ ]:
db.search('treatment of scrap steel fly ash')[0]['code']